In [5]:
import pandas as pd
import numpy as np

In [6]:
listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)

In [7]:
listing_details.shape[0]

32461

In [234]:
listing_details_cleaned = pd.read_csv('output/listing_details_cleaned.csv')
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27246 entries, 0 to 27245
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27246 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27246 non-null  object 
 2   Xã/Phường/Thị trấn                    26947 non-null  object 
 3   Đường phố                             20229 non-null  object 
 4   Chi tiết                              27246 non-null  object 
 5   Nguồn thông tin                       27246 non-null  object 
 6   Tình trạng giao dịch                  27246 non-null  object 
 7   Thời điểm giao dịch/rao bán           27236 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24742 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27246 non-null  object 
 11  Số tầng công tr

In [235]:
listing_details_cleaned.rename(columns={'Nguồn thông tin': 'url'}, inplace=True)
listing_details_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27246 entries, 0 to 27245
Data columns (total 29 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27246 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27246 non-null  object 
 2   Xã/Phường/Thị trấn                    26947 non-null  object 
 3   Đường phố                             20229 non-null  object 
 4   Chi tiết                              27246 non-null  object 
 5   url                                   27246 non-null  object 
 6   Tình trạng giao dịch                  27246 non-null  object 
 7   Thời điểm giao dịch/rao bán           27236 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24742 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27246 non-null  object 
 11  Số tầng công tr

In [236]:
cleaned = pd.merge(listing_details_cleaned, listing_details, how='left', on='url')
cleaned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27248 entries, 0 to 27247
Data columns (total 35 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27248 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27248 non-null  object 
 2   Xã/Phường/Thị trấn                    26949 non-null  object 
 3   Đường phố                             20231 non-null  object 
 4   Chi tiết                              27248 non-null  object 
 5   url                                   27248 non-null  object 
 6   Tình trạng giao dịch                  27248 non-null  object 
 7   Thời điểm giao dịch/rao bán           27238 non-null  object 
 8   Thông tin liên hệ                     0 non-null      float64
 9   Giá rao bán/giao dịch                 24744 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27248 non-null  object 
 11  Số tầng công tr

## Columns to fix: 

- Thành phố/Quận/Huyện/Thị xã (Xong)
- Xã/Phường/Thị trấn (xong)

Đường phố, Giá rao bán/giao dịch, Số tầng công trình, Tổng diện tích sàn, Đơn giá xây dựng, Chất lượng còn lại, Diện tích đất (m2), Kích thước mặt tiền (m), Kích thước chiều dài (m), Số mặt tiền tiếp giáp, Hình dạng, Độ rộng ngõ/ngách nhỏ nhất (m), Khoảng cách tới trục đường chính (m)

# Test district from here

In [1]:
import argparse
import os
from datetime import date
import threading
from concurrent.futures import ThreadPoolExecutor, as_completed

import pandas as pd
import numpy as np

from src.selenium_manager import create_stealth_driver
from src.scraping import Scraper
from src.cleaning import DataCleaner, drop_mixed_listings, is_land_only
from src.feature_engineering import FeatureEngineer
from src.address_standardizer import AddressStandardizer
from src import config
from src.utils import save_urls_to_csv, save_details_to_csv, chunks
from src.tasks import scrape_worker
from src.modelling import predict_alley_width

def run_cleaning_pipeline():
    """Step 3: Clean the raw data and structure it."""
    if not os.path.exists(config.DETAILS_OUTPUT_FILE):
        print(f"Raw details file not found: {config.DETAILS_OUTPUT_FILE}. Run with `--mode details` first.")
        return

    print(f"Reading raw data from '{config.DETAILS_OUTPUT_FILE}'...")
    df_raw = pd.read_csv(config.DETAILS_OUTPUT_FILE)
    df_raw = drop_mixed_listings(df_raw)

    cleaned_records = []
    for _, row in df_raw.iterrows():
        row_dict = row.to_dict()
        direct_features = DataCleaner.extract_direct_features(row_dict)

        # --- 1. Detect if it is a land-only property ---
        is_land = is_land_only(row_dict)

        # --- 2. Extract all data ---
        processed_data = {
            'Tỉnh/Thành phố': DataCleaner.extract_city(row_dict),
            'Thành phố/Quận/Huyện/Thị xã': DataCleaner.extract_district(row_dict),
            'Xã/Phường/Thị trấn': DataCleaner.extract_ward(row_dict),
            'Đường phố': DataCleaner.extract_street(row_dict),
            'Chi tiết': DataCleaner.extract_address_detail(row_dict),
            'Nguồn thông tin': row_dict.get('url'),
            'Tình trạng giao dịch': 'Rao bán',
            'Thời điểm giao dịch/rao bán': DataCleaner.extract_published_date(row_dict.get('main_info')),
            'Thông tin liên hệ': None,
            'Giá rao bán/giao dịch': DataCleaner.extract_total_price(row_dict.get('main_info')),
            'Loại đơn giá (đ/m2 hoặc đ/m ngang)': 'đ/m2',
            'Số tầng công trình': DataCleaner.extract_num_floors(row_dict),
            'Tổng diện tích sàn': DataCleaner.extract_built_area(row_dict),
            'Đơn giá xây dựng': DataCleaner.get_construction_cost(row_dict),
            'Năm xây dựng': None,
            'Chất lượng còn lại': DataCleaner.estimate_remaining_quality(row_dict),
            'Diện tích đất (m2)': DataCleaner.extract_total_area(row_dict),
            'Kích thước mặt tiền (m)': DataCleaner.extract_facade_width(row_dict),
            'Kích thước chiều dài (m)': DataCleaner.extract_land_length(row_dict),
            'Số mặt tiền tiếp giáp': DataCleaner.extract_facade_count(row_dict),
            'Hình dạng': DataCleaner.extract_land_shape(row_dict),
            'Độ rộng ngõ/ngách nhỏ nhất (m)': DataCleaner.extract_alley_width(row_dict),
            'Khoảng cách tới trục đường chính (m)': DataCleaner.extract_distance_to_main_road(row_dict),
            'Mục đích sử dụng đất': 'Đất ở',
            'Yếu tố khác': " | ".join(direct_features) if direct_features else None,
            'Tọa độ (vĩ độ)': row_dict.get('latitude'),
            'Tọa độ (kinh độ)': row_dict.get('longitude'),
            'Hình ảnh của bài đăng': row_dict.get('image_urls'),
            'description': row_dict.get('description'),
            'is_land': is_land  # <-- Add the temporary flag here
        }

        # --- 3. Apply special logic if it's land only ---
        if is_land:
            processed_data['Số tầng công trình'] = 0
            processed_data['Đơn giá xây dựng'] = 0
            processed_data['Tổng diện tích sàn'] = 0
            processed_data['Chất lượng còn lại'] = 0
        
        cleaned_records.append(processed_data)

    df_cleaned = pd.DataFrame(cleaned_records)

    try:
        # Standardize Province and District using the simplified AddressStandardizer
        address_std = AddressStandardizer(
            config.PROVINCES_SQL_FILE,
            config.DISTRICTS_SQL_FILE,
            config.WARDS_SQL_FILE,
            config.STREETS_SQL_FILE
        )
        df_cleaned['Tỉnh/Thành phố'] = df_cleaned['Tỉnh/Thành phố'].apply(address_std.standardize_province)
        df_cleaned['short_address'] = df_raw['short_address']
        df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        df_cleaned['Xã/Phường/Thị trấn'] = df_cleaned.apply(address_std.standardize_ward, axis = 1)
        df_cleaned.drop(columns=['short_address'], inplace = True)
        return df_cleaned
        # df_cleaned['Thành phố/Quận/Huyện/Thị xã'] = df_cleaned.apply(address_std.standardize_district, axis=1)
        # df_cleaned.drop(columns=['short_address'], inplace=True)
        # print("Province and District standardization complete.")
    except FileNotFoundError:
        print("Skipping province/district standardization because data files were not found.")

cleaned = run_cleaning_pipeline()

Reading raw data from 'output/listing_details.csv'...
Removed 4972 listings containing 'thổ cư'.
Error parsing price: local variable 'cleaned_num' referenced before assignment


In [2]:
cleaned['Xã/Phường/Thị trấn'].isna().sum()

np.int64(22)

In [9]:
# from src.address_standardizer import AddressStandardizer
from src import config
import sqlite3
import pandas as pd
from unicodedata import normalize
from rapidfuzz import fuzz

conn = sqlite3.connect(":memory:")
conn.execute("CREATE TABLE provinces (name TEXT, code TEXT, status TEXT);")
conn.execute("CREATE TABLE districts (name TEXT, code TEXT, province_code TEXT, status TEXT);")
conn.execute("CREATE TABLE wards (name TEXT, code TEXT, district_code TEXT, status TEXT);")
conn.execute("CREATE TABLE streets (name TEXT, code TEXT, district_code TEXT, status TEXT);")

with open(config.PROVINCES_SQL_FILE, "r", encoding="utf-8") as f:
    conn.executescript(f.read())

with open(config.DISTRICTS_SQL_FILE, "r", encoding="utf-8") as f:
    dis_cleaned = f.read().replace("\\'", "''")
    conn.executescript(dis_cleaned)

with open(config.WARDS_SQL_FILE, "r", encoding="utf-8") as f:
    ward_cleaned = f.read().replace("\\'", "''")
    conn.executescript(ward_cleaned)

provinces_df = pd.read_sql_query("SELECT * FROM provinces", conn)
districts_df = pd.read_sql_query("""
    SELECT d.name AS district_name, p.name AS province_name
    FROM districts d
    JOIN provinces p ON d.province_code = p.code
    """, conn)

wards_df = pd.read_sql_query("""
    SELECT w.name AS ward_name,
        d.name AS district_name,
        p.name AS province_name
    FROM wards w
    JOIN districts d ON w.district_code = d.code
    JOIN provinces p ON d.province_code = p.code
""", conn)

if 'conn' in locals():
    conn.close()

reverse_province_map = {
    prov.replace("Thành phố ", "").replace("Tỉnh ", ""): prov
    for prov in provinces_df['name'].unique()
}

reverse_district = {}
for province in districts_df['province_name'].unique():
    reverse_district[province] = {}
    for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
        district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
        reverse_district[province][district_name_strip] = district_name
reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'
reverse_district['Thành phố Hồ Chí Minh']['Quận 2'] = 'Thành phố Thủ Đức'
reverse_district['Thành phố Hồ Chí Minh']['Quận 9'] = 'Thành phố Thủ Đức'

# for province in districts_df['province_name'].unique():
#     reverse_district[province] = {}
#     for district_name in districts_df[districts_df['province_name'] == province]['district_name'].unique():
#         # district_name_strip = normalize('NFKD', district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip())
#         district_name_strip = district_name.replace('Thành phố ', '').replace('Thành Phố ', '').replace('Quận ', '').replace('Huyện ', '').replace('Thị xã ', '').replace('Thị Xã ', '').strip()
#         reverse_district[province][district_name_strip] = district_name
# reverse_district['Tỉnh Bà Rịa - Vũng Tàu']['Long Đất'] = 'Huyện Long Đất'

reverse_ward = {}
for province in reverse_district.keys():
    reverse_ward[province] = {}
    for district in reverse_district[province].values():
        reverse_ward[province][district] = {}
        for ward in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
            ward_name_strip = normalize('NFC', ward.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
            reverse_ward[province][district][ward_name_strip] = ward
# for district in wards_df['district_name'].unique():
#     reverse_ward[district] = {}
#     for ward_name in wards_df[wards_df['district_name'] == district]['ward_name'].unique():
#         ward_name_strip = normalize('NFKD', ward_name.replace('Xã ', '').replace('Phường ', '').replace('Thị trấn ', '').replace('Thị Trấn ', '').strip())
#         reverse_ward[district][ward_name_strip] = ward_name

def standardize_district(row):
        prefix = ['Thành phố', 'Thành Phố', 'Quận', 'Huyện', 'Thị xã', 'Thị Xã', 'Đảo']
        district_value = row['Thành phố/Quận/Huyện/Thị xã']
        if isinstance(district_value, str):
            for pre in prefix:
                if district_value.startswith(pre):
                    return district_value
            province = row['Tỉnh/Thành phố']
            if district_value in reverse_district[province].keys():
                return reverse_district[province][district_value]
            for dis in reverse_district[province].keys():
                similarity = fuzz.ratio(district_value, dis)
                if similarity >= 66:
                    print(f'Value: {district_value}')
                    print(f"Short address: {row['short_address']}")
                    print(f"Predicted value: {reverse_district[province][dis]}")
                    print('-' * 50)
                    return reverse_district[province][dis]
            return district_value
        return None

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            try:
                if ward_value in reverse_ward[province_value][district_value].keys():
                    return reverse_ward[province_value][district_value][ward_value]
            except:
                print(f'Ward value: {ward_value}\nDistrict value: {district_value}\nProvince value: {province_value}')
                print('-'*50)
                return None
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add_value = row['short_address']
            if isinstance(short_add_value, str) and short_add_value != '':
                short_add_list = row['short_address'].split(',')
                if len(short_add_list) >= 3:
                    new_province_val = row['Tỉnh/Thành phố']
                    new_ward_val = normalize('NFC',short_add_list[-3].strip())
                    new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                    return matching(new_ward_val, new_district_val, new_province_val)
            return None


# cleaned['ward'] = cleaned.apply(standardize_ward, axis=1)

# Test Ward

In [10]:
from unicodedata import normalize
import re

def standardize_ward(row):
        ward_value = row['Xã/Phường/Thị trấn']

        def matching(ward_value, district_value, province_value):
            # Function to match values with its corresponding prefixes
            if ward_value in reverse_ward[province_value][district_value].keys():
                return reverse_ward[province_value][district_value][ward_value]
            for ward in reverse_ward[province_value][district_value].keys():
                similarity = fuzz.ratio(ward_value, ward)
                if similarity >= 66:
                    return reverse_ward[province_value][district_value][ward]
            return None
                
        if ward_value:
            prefix = ['Xã', 'Phường', 'Thị trấn', 'Thị Trấn']
            for pre in prefix:
                if ward_value.startswith(pre):
                    return ward_value
            ward_value = normalize('NFC', ward_value)
            province_value = row['Tỉnh/Thành phố']
            district_value = row['Thành phố/Quận/Huyện/Thị xã']
            return matching(ward_value, district_value, province_value)
        else:
            short_add = row['short_address']
            if isinstance(short_add, str) and short_add != '':
                if 'xã' in short_add.lower():
                    print(f"Xã in short_add: {short_add.lower()}")
                    match_result = re.search(pattern='(xã [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'phường' in short_add.lower():
                    print(f'Phường in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(phường [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                elif 'thị trấn' in short_add.lower():
                    print(f'Thị trấn in short_add: {short_add.lower()}')
                    match_result = re.search(pattern='(thị trấn [\w\s]+)', string=short_add.lower())
                    if match_result:
                        match_result = match_result[0]
                        result_split = match_result.split()
                        result = ' '.join(i.capitalize() for i in result_split)
                        return result
                else:
                    short_add_list = row['short_address'].split(',')
                    if len(short_add_list) >= 3:
                        new_province_val = row['Tỉnh/Thành phố']
                        new_ward_val = normalize('NFC',short_add_list[-3].strip())
                        new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                        return matching(new_ward_val, new_district_val, new_province_val)

            else:
                return None
                # short_add_list = row['short_address'].split(',')
                # if len(short_add_list) >= 3:
                #     new_province_val = row['Tỉnh/Thành phố']
                #     new_ward_val = normalize('NFC',short_add_list[-3].strip())
                #     new_district_val = row['Thành phố/Quận/Huyện/Thị xã']
                #     return matching(new_ward_val, new_district_val, new_province_val)
            return None

In [20]:
from rapidfuzz import fuzz

yo = cleaned['district'].iloc[24895]
dis = list(reverse_district['Tỉnh Đắk Lắk'].keys())[4]
print(fuzz.ratio(yo, dis))

66.66666666666667


# Test Facade Width (Kích thước mặt tiền)

In [15]:
import pandas as pd
import numpy as np

listing_details = pd.read_csv('output/listing_details.csv')
listing_details.drop(['latitude', 'longitude', 'image_urls'], axis=1, inplace=True)
cleaned.rename(columns={"Nguồn thông tin": 'url'}, inplace=True)

df = pd.merge(left=cleaned, right=listing_details, how='left', on='url')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27491 entries, 0 to 27490
Data columns (total 36 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Tỉnh/Thành phố                        27491 non-null  object 
 1   Thành phố/Quận/Huyện/Thị xã           27491 non-null  object 
 2   Xã/Phường/Thị trấn                    27469 non-null  object 
 3   Đường phố                             23802 non-null  object 
 4   Chi tiết                              27491 non-null  object 
 5   url                                   27491 non-null  object 
 6   Tình trạng giao dịch                  27491 non-null  object 
 7   Thời điểm giao dịch/rao bán           27479 non-null  object 
 8   Thông tin liên hệ                     0 non-null      object 
 9   Giá rao bán/giao dịch                 24953 non-null  float64
 10  Loại đơn giá (đ/m2 hoặc đ/m ngang)    27491 non-null  object 
 11  Số tầng công tr

In [20]:
df['other_info']

0        {"Mức giá": "2,8 tỷ", "Diện tích": "48 m²", "S...
1        {"Mức giá": "5,7 tỷ", "Diện tích": "52 m²", "S...
2        {"Mức giá": "10,5 tỷ", "Diện tích": "64 m²", "...
3        {"Mức giá": "18,2 tỷ", "Diện tích": "82 m²", "...
4        {"Mức giá": "Thỏa thuận", "Diện tích": "62 m²"...
                               ...                        
27486    {"Mức giá": "780 triệu", "Diện tích": "100 m²"...
27487    {"Mức giá": "17,2 tỷ", "Diện tích": "80 m²", "...
27488     {"Mức giá": "934 triệu", "Diện tích": "92,6 m²"}
27489    {"Mức giá": "4,85 tỷ", "Diện tích": "90 m²", "...
27490    {"Mức giá": "4,65 tỷ", "Diện tích": "246,2 m²"...
Name: other_info, Length: 27491, dtype: object